In [ ]:
import requests
from bs4 import BeautifulSoup as bs
import numpy as np
import matplotlib.pyplot as plt
import concurrent.futures
import pandas as pd
import re
from threading import Thread
import collections
import json
from datetime import datetime as dt
from fuzzywuzzy import fuzz

In [7]:
df = pd.read_csv('../data/processed.csv')
clubs = df['Club'].unique()

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17028 entries, 0 to 17027
Data columns (total 47 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Name                 17028 non-null  object 
 1   Overall Rating       17028 non-null  float64
 2   Potential            17028 non-null  float64
 3   Value                17028 non-null  float64
 4   Wage                 17028 non-null  float64
 5   Crossing             17028 non-null  float64
 6   Finishing            17028 non-null  float64
 7   Heading Accuracy     17028 non-null  float64
 8   Short Passing        17028 non-null  float64
 9   Volleys              17028 non-null  float64
 10  Dribbling            17028 non-null  float64
 11  Curve                17028 non-null  float64
 12  FK Accuracy          17028 non-null  float64
 13  Long Passing         17028 non-null  float64
 14  Ball Control         17028 non-null  float64
 15  Acceleration         17028 non-null 

In [19]:
def get_links(clubs,n_pages):
    links = []
    offset = range(0,n_pages*60,60)
    for value in offset:
        url = f'https://sofifa.com/teams?type=club&offset={value}'
        r = requests.get(url)
        soup = bs(r.content)
        page = soup.select('td > a')
        links += ['https://sofifa.com'+page[i]['href'] for i in range(len(page)) if i % 2 == 0]
    
    return [i[0] for i in collections.Counter(links).most_common()]

links = get_links(clubs,20)


In [33]:
result = []
for url in links:
    info = {}
    r = requests.get(url)
    soup = bs(r.content)
    name = soup.find('h1').get_text()
    if name in clubs:
        info['Name'] = name
        for i in soup.select('section.card'):
            nums = re.compile(r'\d+').findall(i.get_text())
            text = re.compile(r'[A-Za-z]+').findall(i.get_text())
            for i in range(len(nums)):
                info[text[i]] = float(nums[i])
    
    result.append(info)

club_df = pd.DataFrame(result)

In [34]:
club_df.head()

,Name,Overall,Attack,Midfield,Defence
0,Liverpool,85.0,86.0,84.0,85.0
1,Manchester City,85.0,84.0,87.0,85.0
2,Paris Saint-Germain,85.0,90.0,82.0,84.0
3,Chelsea,84.0,83.0,86.0,83.0
4,FC Bayern München,84.0,92.0,85.0,82.0


In [38]:
club_df.dropna(inplace=True)

In [39]:
club_df.to_csv('../data/club_data.csv',index=False)